In [ ]:
# 아래 코드는 jupyter notebook에서 실행하는 데 필요한 코드.
# !pip install pandas scipy matplotlib seaborn
import os
# 현재 작업 중인 디렉토리 확인
print(os.getcwd())

# 작업 디렉토리 변경
# abundance, metadata가 있는 파일이 있는 경로로 지정.
os.chdir("H:/experiments_ing/ML_BC_relapse/relative_abundance_statistics")

print(os.getcwd())

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kruskal

# 1. 데이터 불러오기
abundance_df = pd.read_csv("abundance_table.csv", index_col=0)
metadata_df = pd.read_csv("metadata.csv", index_col=0)

# 2. 상대적 풍부도 계산
relative_abundance_df = abundance_df.div(abundance_df.sum(axis=0), axis=1)

print(relative_abundance_df.head())

# 데이터의 첫 몇 행을 확인
print(abundance_df.head())

# 데이터의 크기 확인
print(abundance_df.shape)

# 10% 이상의 샘플에서 관찰된 미생물만 유지
threshold = 0.1
presence_threshold = (abundance_df > 0).mean(axis=1) > threshold

# 필터링된 데이터프레임
filtered_abundance_df = abundance_df.loc[presence_threshold]

print(f"필터링 전 미생물 개수: {abundance_df.shape[0]}")
print(f"필터링 후 미생물 개수: {filtered_abundance_df.shape[0]}")

# 필터링된 데이터 확인
print(filtered_abundance_df.head())

import seaborn as sns
import matplotlib.pyplot as plt

# 상대적 풍부도 계산
relative_abundance_df = filtered_abundance_df.div(filtered_abundance_df.sum(axis=0), axis=1)

# 특정 미생물의 상대적 풍부도 시각화 (예: 상위 10개 미생물)
top_microbes = relative_abundance_df.mean(axis=1).sort_values(ascending=False).head(10).index

plt.figure(figsize=(14, 7))
sns.boxplot(data=relative_abundance_df.loc[top_microbes].T)
plt.xticks(rotation=90)
plt.title('Top 10 Microbes Relative Abundance after Filtering')
plt.show()

import pandas as pd
import scipy.stats as stats

# 데이터프레임의 첫 몇 행을 확인
print(abundance_df.head())

# 데이터의 크기 확인
print(abundance_df.shape)

# 그룹 정보가 담긴 데이터프레임 (예: 그룹 정보가 첫 번째 행 또는 별도의 시리즈로 주어진 경우)

# 실제 데이터에 맞춰서 그룹을 지정
group_labels = ['healthy'] * 141 + ['control'] * 81 + ['relapse'] * 7

# 그룹 정보가 담긴 Series 생성
group_df = pd.Series(data=group_labels, index=abundance_df.columns)

print(group_df.head())

# p-value 계산을 위한 함수 정의 (ANOVA 사용)
def calculate_anova_p_value(row):
    # 각 그룹별로 값을 분리하고 NaN을 제거
    healthy_values = row[group_df[group_df == 'healthy'].index].dropna()
    control_values = row[group_df[group_df == 'control'].index].dropna()
    relapse_values = row[group_df[group_df == 'relapse'].index].dropna()

    # 그룹별로 값이 모두 NaN인 경우, p-value를 NaN으로 반환
    if healthy_values.empty or control_values.empty or relapse_values.empty:
        return float('nan')

    # ANOVA 검정 수행 (NaN 값은 dropna()로 제거됨)
    f_stat, p_val = stats.f_oneway(healthy_values, control_values, relapse_values)
    return p_val


# 각 미생물에 대해 p-value 계산
p_values = abundance_df.apply(calculate_anova_p_value, axis=1)

# p-values를 데이터프레임에 추가
abundance_df['p_value'] = p_values

# p-value 임계값 설정 (예: 0.05)
p_value_threshold = 0.05

# 10% 이상의 샘플에서 관찰된 미생물만 유지하고, p-value가 임계값 이하인 경우만 필터링
presence_threshold = (abundance_df.drop(columns=['p_value']) > 0).mean(axis=1) > threshold
p_value_filter = abundance_df['p_value'] <= p_value_threshold
filtered_abundance_df = abundance_df.loc[presence_threshold & p_value_filter]

print(f"필터링 전 미생물 개수: {abundance_df.shape[0]}")
print(f"필터링 후 미생물 개수: {filtered_abundance_df.shape[0]}")

# 필터링된 데이터 확인
print(filtered_abundance_df.head())

# 필터링된 데이터를 엑셀 파일로 저장
# 파일명과 경로를 지정합니다. 예: "filtered_abundance_data.xlsx"
output_file_path = "filtered_abundance_data.xlsx"

# 엑셀 파일로 내보내기
filtered_abundance_df.to_excel(output_file_path, index=True)  # index=True는 행 인덱스를 엑셀 파일에 포함시킵니다.

print(f"필터링된 데이터가 '{output_file_path}'에 저장되었습니다.")

